In [1]:
import pandas as pd

import google.generativeai as genai
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

genai.configure(api_key=API_KEY)
headers = {'Content-Type': 'application/json'}
model = genai.GenerativeModel('gemini-pro')

In [3]:
def dataReader(fileName):
    df = pd.read_csv(fileName)
    df_str = df.to_string()
    return df_str

優化 AI 回應可以嘗試：
1. 加入明確的指標（e.g. 「EPS > 1.0 的股票」）
2. 給 AI 範例輸出（讓ai模仿）

In [4]:
history = []
def aiAdvisor(df_str):
    global history
    
    prompting = f"""
    你是一位專業的投資顧問，專注於低頻交易策略。
    請根據以下財務數據，為投資者設計合適的低頻交易策略：
    1. 價值投資策略：根據「營收」、「本期淨利」、「基本每股盈餘（EPS）」等數據，評估是否有低估的股票適合長期持有。
    2. 動量交易策略：如果某些公司連續幾季「營收」或「淨利潤」增長，請標記出來，作為潛在的投資標的。
    3. 低波動選股策略：挑選「基本每股盈餘」和「本期淨利」較穩定的公司，適合低風險投資者持有。

    請提供你的分析結果，包括：
    - 你認為值得關注的公司及其指標
    - 該公司適用的策略（價值投資、動量交易、低波動選股）
    - 具體的投資建議

    以下是最新的財務數據：
    {df_str}
    """
    
    history.append({"role": "user", "content": prompting})
    
    response = model.generate_content(
        prompting,
        generation_config=genai.types.GenerationConfig(max_output_tokens=800,temperature=0.5)
    )
    
    history.append({"role": "assistant", "content": response.text})

    return response.text

In [1]:
def getHistory():
    return history

In [9]:
def promptImprovement(original_prompt, score):
    
    improvement_prompt = f"""
    你是一位 Prompt Engineering 專家，以下是目前的 Prompt：
    {original_prompt}

    使用者給出的評價是 {score} 分（1-5 分，5 分為最滿意）。
    請根據該評價分數，對原先的prompt重新優化，讓它更適合財務數據分析，並解釋修改的地方。
    """
    improved_prompt = model.generate_content(improvement_prompt)
    return improved_prompt.text

In [10]:
prompting = """
你是一位專業的投資顧問，專注於低頻交易策略。
請根據以下財務數據，為投資者設計合適的低頻交易策略：
1. 價值投資策略：根據「營收」、「本期淨利」、「基本每股盈餘（EPS）」等數據，評估是否有低估的股票適合長期持有。
2. 動量交易策略：如果某些公司連續幾季「營收」或「淨利潤」增長，請標記出來，作為潛在的投資標的。
3. 低波動選股策略：挑選「基本每股盈餘」和「本期淨利」較穩定的公司，適合低風險投資者持有。

請提供你的分析結果，包括：
- 你認為值得關注的公司及其指標
- 該公司適用的策略（價值投資、動量交易、低波動選股）
- 具體的投資建議

以下是最新的財務數據：
"""
newPrompt = promptImprovement(prompting, 3)
print(newPrompt)

**重新優化的 Prompt：**

你是一位專業的投資顧問，專注於低頻交易策略。

請根據以下財務數據，為投資者設計合適的低頻交易策略，並提供具體的投資建議：

- 價值投資策略：評估「營收」、「本期淨利」、「基本每股盈餘（EPS）」等數據是否有低估的股票適合長期持有。進一步分析公司的財務報表，包括資產負債表、損益表和現金流量表，以找出財務穩健且具有成長潛力的公司。

- 動量交易策略：標記連續幾季「營收」或「淨利潤」增長的公司，作為潛在的投資標的。使用技術指標，例如移動平均線和相對強度指數 (RSI)，以確認趨勢的強度和持續性。

- 低波動選股策略：挑選「基本每股盈餘」和「本期淨利」較穩定的公司，適合低風險投資者持有。計算標準差或β值以量化波動性。此外，考慮公司的行業和競爭環境，以評估潛在的風險和回報。

**具體的投資建議：**

- 為每間公司提供明確的投資建議，包括買入、持有或賣出的建議，以及目標價格。
- 解釋投資建議的依據，包括所使用的指標、技術分析結果和基本面分析的見解。

**解釋：**

原先的提示過於籠統，沒有明確說明如何使用提供的財務數據。重新優化的提示增加了具體的指標和分析方法，以提高分析和投資建議的品質。此外，進一步的分析，例如財務報表的審查和技術指標的使用，將有助於提供更深入的見解和更可信的建議。


In [5]:
data_1 = dataReader("final_result_1.csv")

a = aiAdvisor(data_1)
print(a)

**分析結果：**

**1. 價值投資策略**

* **宏遠證 (6015)**
    * 本期淨利：352,180,000 元
    * 基本每股盈餘：0.96 元
    * 股價：13.90 元
    * 本益比：約 14.5 倍

宏遠證的本益比低於同業平均，且基本每股盈餘穩定，符合價值投資策略的標準。

**2. 動量交易策略**

* **康和證 (6016)**
    * 營收：連續三季成長
    * 淨利潤：連續三季成長
    * 股價：15.47 元

康和證的營收和淨利潤持續成長，符合動量交易策略的條件。

**3. 低波動選股策略**

* **大展證 (6020)**
    * 基本每股盈餘：0.82 元
    * 本期淨利：208,006,000 元
    * 股價：18.70 元
    * 本益比：約 22.8 倍

大展證的基本每股盈餘和本期淨利較為穩定，符合低波動選股策略的標準。

**投資建議：**

* **價值投資策略：**長期持有宏遠證 (6015)
* **動量交易策略：**逢低買進康和證 (6016)
* **低波動選股策略：**長期持有大展證 (6020)
